In [1]:
import gzip
import pickle
from tensorflow import keras
import numpy as np
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, BatchNormalization, \
                                    Activation, Dropout, MaxPooling2D

def evaluate(path, model):
    X, y = pickle.load(gzip.open(path, 'rb'))
    y[y != 0] -= 2
    X = X / 255.
    acc = np.mean(model(X).numpy().argmax(axis=1) == y)
    return acc


In [2]:
path = 'flatland_train.data'
X,y = pickle.load(gzip.open(path,'rb')) 

y[y != 0] -= 2
X = X.reshape (X.shape[0],50,50,1) / 255.

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 42)

DNN modelis

In [5]:
# model = keras.models.Sequential()
# model.add(keras.layers.Flatten(input_shape=[50, 50]))
# model.add(keras.layers.Dense(64, activation="relu"))
# model.add(keras.layers.Dense(32, activation="relu"))
# model.add(keras.layers.Dense(10, activation="softmax"))
# model.compile(loss="sparse_categorical_crossentropy",
#               optimizer="Adam",
#               metrics=["accuracy"])
# model.summary()

CNN modelis

In [4]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='softmax', input_shape=[50, 50, 1]))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='softmax'))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(keras.layers.Conv2D(128, kernel_size=(3, 3), activation='softmax'))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(keras.layers.Conv2D(256, kernel_size=(3, 3), activation='softmax'))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))

model.add(Dropout(0.5))

model.add(Flatten())
model.add(keras.layers.Dense(60, activation="relu"))
model.add(keras.layers.Dense(53, activation="relu"))

model.add(keras.layers.Dense(15, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="Adam",
              metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 22, 22, 64)       256       
 hNormalization)                                                 
                                                        

In [5]:
loss = model.fit(X_train, y_train, epochs=7,validation_data=[X_test,y_test])


Epoch 1/7
219/219 [==============================] - 7s 20ms/step - loss: 1.2750 - accuracy: 0.5143 - val_loss: 1.8635 - val_accuracy: 0.1830
Epoch 2/7
219/219 [==============================] - 4s 17ms/step - loss: 0.5186 - accuracy: 0.8101 - val_loss: 1.8176 - val_accuracy: 0.2097
Epoch 3/7
219/219 [==============================] - 4s 17ms/step - loss: 0.2789 - accuracy: 0.9223 - val_loss: 1.0832 - val_accuracy: 0.6223
Epoch 4/7
219/219 [==============================] - 4s 17ms/step - loss: 0.2053 - accuracy: 0.9490 - val_loss: 0.1516 - val_accuracy: 0.9793
Epoch 5/7
219/219 [==============================] - 4s 17ms/step - loss: 0.1699 - accuracy: 0.9619 - val_loss: 0.2368 - val_accuracy: 0.9360
Epoch 6/7
219/219 [==============================] - 4s 17ms/step - loss: 0.1455 - accuracy: 0.9694 - val_loss: 0.1587 - val_accuracy: 0.9783
Epoch 7/7
219/219 [==============================] - 4s 17ms/step - loss: 0.1373 - accuracy: 0.9701 - val_loss: 0.1676 - val_accuracy: 0.9733


In [8]:
model.save('model_cnn24.h5')


In [ ]:
model = keras.models.load_model('model_cnn24.h5')
evaluate('flatland_train.data', model)

In [10]:
pred = model.predict(X_test).argmax(axis=1)
print('Accuracy on test set - {0:.02%}'.format((pred == y_test).mean()))

Accuracy on test set - 97.43%
